In [ ]:
#예제 9.1 FastFeatureDetector 특징 검출 1
import cv2
import numpy as np

src = cv2.imread('0205.png\chessboard.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

#1
##fastF = cv2.FastFeatureDetector.create
##fastF = cv2.FastFeatureDetector_create
fastF = cv2.FastFeatureDetector.create(threshold = 30)  #100 #임계값 30인 클래스 객체 fastF 생성
kp = fastF.detect(gray) #gray에서 특징점 검출
dst = cv2.drawKeypoints(gray, kp, None, color = (0, 0, 255)) #특징점 kp를 dst에 표시
print('len(kp)=', len(kp))
cv2.imshow('dst', dst)

#2
fastF.setNonmaxSuppression(False) #지역 극값 억제x
kp2 = fastF.detect(gray)
dst2 = cv2.drawKeypoints(src, kp2, None, color = (0, 0, 255))
print('len(kp2)=', len(kp2))
cv2.imshow('dst2', dst2)

#3
dst3 = src.copy()
points = cv2.KeyPoint_convert(kp)
points = np.int32(points)

for cx, cy in points:
    cv2.circle(dst3, (cx, cy), 3, color = (255, 0, 0), thickness= 1)
cv2.imshow('dst3', dst3)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 9.2 FastFeatureDetector 특징 검출 2: 특징 정렬, 필터링
import cv2
import numpy as np

src = cv2.imread('0205.png\chessboard.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

#1
fastF = cv2.FastFeatureDetector_create()  #기본 임계값 10
kp = fastF.detect(gray) #gray에서 특징점 검출
dst = cv2.drawKeypoints(gray, kp, None, color = (255, 0, 0)) #특징점 kp를 dst에 표시
print('len(kp)=', len(kp))

#2
kp = sorted(kp, key = lambda f: f.response, reverse = True) #kp를 내림차순으로 정렬
cv2.drawKeypoints(gray, kp[:10], dst, color = (0, 0, 255),
                  flags= cv2.DRAW_MATCHES_FLAGS_DRAW_OVER_OUTIMG) #반응값이 큰 10개를 빨간색 원으로 표시
cv2.imshow('dst', dst)

#3
kp2 = list(filter(lambda f: f.response > 50, kp)) #반응값이 50보다 작은 특징은 제거
print('len(kp2)=', len(kp2))
##for f in kp2:
##    print(f.response)

dst2 = cv2.drawKeypoints(gray, kp2, None, color= (0, 0, 255))
cv2.imshow('dst2', dst2)

#4
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def filteringByDistance(kp, distE = 0.5): #내림차순 정렬된 특징점에서 거리오차 distE보다 작은 특징점은 삭제
    size = len(kp)
    mask = np.arange(1, size + 1).astype(np.bool8)  # all True
    for i, f1 in enumerate(kp):
        if not mask[i]:
            continue
        else:   #True
            for j, f2 in enumerate(kp):
                if i == j:
                    continue
                if distance(f1, f2) < distE:
                    mask[j] = False
        np_kp = np.array(kp)
        return list(np_kp[mask])

kp3 = filteringByDistance(kp2, 30)
print('len(kp3)=', len(kp3))
dst3 = cv2.drawKeypoints(gray, kp3, None, color = (0, 0, 255))
cv2.imshow('dst3', dst3)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 9.3 MSER 특징 검출
import cv2
import numpy as np

src = cv2.imread('0205.png\chessboard.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

#1
mserF = cv2.MSER_create(10) #안정적인 그레이 레벨의 단계의 간격 _delta=10    #cv2.MSER.create(10)
kp = mserF.detect(gray) #영역의 중심점인 특징점 kp 검출
print('len(kp)=', len(kp))
dst = cv2.drawKeypoints(gray, kp, None, color = (0, 0, 255))
cv2.imshow('dst', dst)

#2
dst2 = dst.copy()
regions, bboxes = mserF.detectRegions(gray) # 특징영역의 좌표를 regions에 검출
hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions] # regions[i] 영역을 볼록다각형 hulls[i]에 계산
cv2.polylines(dst2, hulls, True, (0, 255, 0))
cv2.imshow('dst2', dst2)

#3
dst3 = dst.copy()
for i, pts in enumerate(regions):
    box = cv2.fitEllipse(pts)  
    cv2.ellipse(dst3, box, (255, 0, 0), 1)
    x, y, w, h = bboxes[i]
    cv2.rectangle(dst3, (x, y), (x + 2, y + h), (0, 255, 0))
cv2.imshow('dst3', dst3)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 9.4 SimpleBlobDetector 특징 검출
import cv2
import numpy as np

src = cv2.imread('0205.png\chessboard.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)


#1 params 객체 생성, 속성을 설정
params = cv2.SimpleBlobDetector_Params()
params.blobColor = 0
params.thresholdStep = 5
params.minThreshold = 20
params.maxThreshold = 100
params.minDistBetweenBlobs = 5
params.filterByArea = True
params.minArea = 25
params.maxArea = 5000
params.filterByConvexity = True
params.minConvexity = 0.89

#2
##blobF = cv2.SimpleBlobDetector.create(params)
##blobF = cv2.SimpleBlobDetector_create(params)
blobF = cv2.SimpleBlobDetector_create()
kp = blobF.detect(gray) #특징점 kp 검출
print('len(kp)=', len(kp))
dst = cv2.drawKeypoints(gray, kp, None, color= (0, 0, 255))

#3
for f in kp:
    r = int(f.size/2)
    cx, cy = f.pt
    cv2.circle(dst, (round(cx), round(cy)), r, (0, 0, 255), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 9.5 GFTTDetector 특징 검출
import cv2
import numpy as np

src = cv2.imread('0205.png\chessboard.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)


#1 객체 goodF 생성
##goodF = cv2.GFTTDetector.create()
goodF = cv2.GFTTDetector_create()
kp = goodF.detect(gray)
print('len(kp)=', len(kp))
dst = cv2.drawKeypoints(gray, kp, None, color = (0, 0, 255))
cv2.imshow('dst', dst)

#2
goodF2 = cv2.GFTTDetector_create(maxCorners= 50,
                                 qualityLevel= 0.1,
                                 minDistance= 10,
                                 useHarrisDetector= True)
kp2 = goodF2.detect(gray)
print('len(kp2)=', len(kp2))
dst2 = cv2.drawKeypoints(gray, kp2, None, color = (0, 0, 255))
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 9.6 ORB 특징 검출 및 디스크립터
import cv2
import numpy as np
#1
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def filteringByDistance(kp, distE=0.5):
    size = len(kp)
    mask = np.arange(1,size + 1).astype(np.bool8)   #all True
    for i, f1 in enumerate(kp):
        if not mask[i]:
            continue
        else: # True
            for j, f2 in enumerate(kp):
                if i == j:
                    continue
                if distance(f1, f2) < distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

#2
src = cv2.imread('0205.png\cv2.preCornerDetect().jpg')
##src = cv2.imread('')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0.0)

##orbF = cv2.ORB_create()   #HARRIS_SCORE
orbF = cv2.ORB_create(scoreType= 1) #FAST_SCORE
kp = orbF.detect(gray)
print('len(kp)=', len(kp))
dst = cv2.drawKeypoints(gray, kp, None, color = (0, 0, 255))
cv2.imshow('dst', dst)

#3
kp = sorted(kp, key = lambda f: f.response, reverse= True)
filtered_kp = list(filter(lambda f:f.response > 50, kp))
filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))

kp, des = orbF.compute(gray, filtered_kp)
print('des.shape=', des.shape)
print('des=', des)

#4
dst2 = cv2.drawKeypoints(gray, filtered_kp, None, color= (0, 0, 255))
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x, y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst2, [box], True, (0, 255, 0), 2)
    cv2.circle(dst2, (round(x), round(y)), round(f.size / 2),
                (255, 0, 0), 2)

cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 9.6 ORB 특징 검출 및 디스크립터
import cv2
import numpy as np
#1 예제 9.2 거리에 따른 kp 제거
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def filteringByDistance(kp, distE=0.5):
    size = len(kp)
    mask = np.arange(1,size + 1).astype(np.bool8)   #all True
    for i, f1 in enumerate(kp):
        if not mask[i]:
            continue
        else: # True
            for j, f2 in enumerate(kp):
                if i == j:
                    continue
                if distance(f1, f2) < distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

#2
src = cv2.imread('0205.png\cv2.preCornerDetect().jpg')
##src = cv2.imread('')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0.0)

##orbF = cv2.ORB_create()   #HARRIS_SCORE
orbF = cv2.ORB_create(scoreType= 1) #FAST_SCORE
kp = orbF.detect(gray)
print('len(kp)=', len(kp))
dst = cv2.drawKeypoints(gray, kp, None, color = (0, 0, 255))
cv2.imshow('dst', dst)

#3
kp = sorted(kp, key = lambda f: f.response, reverse= True)
filtered_kp = list(filter(lambda f:f.response > 50, kp))
filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))

kp, des = orbF.compute(gray, filtered_kp)
print('des.shape=', des.shape)
print('des=', des)

#4
dst2 = cv2.drawKeypoints(gray, filtered_kp, None, color= (0, 0, 255))
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x, y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst2, [box], True, (0, 255, 0), 2)
    cv2.circle(dst2, (round(x), round(y)), round(f.size / 2),
                (255, 0, 0), 2)

cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 9.7 다른 특징 검출기의 ORB 디스크립터
import cv2
import numpy as np

#1 예제 9.2 거리에 따른 kp 제거
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def filteringByDistance(kp, distE=0.5):
    size = len(kp)
    mask = np.arange(1,size + 1).astype(np.bool8)   #all True
    for i, f1 in enumerate(kp):
        if not mask[i]:
            continue
        else: # True
            for j, f2 in enumerate(kp):
                if i == j:
                    continue
                if distance(f1, f2) < distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

#2
src = cv2.imread('0205.png\chessboard.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0.0)

fastF = cv2.FastFeatureDetector_create(threshold= 30)
mserF = cv2.MSER_create(10)
blobF = cv2.SimpleBlobDetector_create()
goodF = cv2.GFTTDetector_create(maxCorners= 20, minDistance= 10)

kp = fastF.detect(gray)
##kp = mserF.detect(gray)
##kp = blobF.detect(gray)
##kp = goodF.detect(gray)
print('len(kp)=', len(kp))

filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))
dst = cv2.drawKeypoints(gray, filtered_kp, None, color= (0, 0, 255))
cv2.imshow('dst', dst)

#3
orbF = cv2.ORB_create()
filtered_kp, des = orbF.compute(gray, filtered_kp) 
print('des.shape=', des.shape)
print('des=', des)

dst2 = cv2.drawKeypoints(gray, filtered_kp, None, color= (0, 0, 255))
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x, y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst2, [box], True, (0, 255, 0), 2)
    cv2.circle(dst2, (round(x), round(y)), round(f.size / 2),
                (255, 0, 0), 2)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 9.8 BRISK 특징 검출 및 디스크립터
import cv2
import numpy as np

#1 예제 9.2 거리에 따른 kp 제거
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def filteringByDistance(kp, distE=0.5):
    size = len(kp)
    mask = np.arange(1,size + 1).astype(np.bool8)   #all True
    for i, f1 in enumerate(kp):
        if not mask[i]:
            continue
        else: # True
            for j, f2 in enumerate(kp):
                if i == j:
                    continue
                if distance(f1, f2) < distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

#2
src = cv2.imread('0205.png\chessboard.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0.0)

briskF = cv2.BRISK_create()
kp = briskF.detect(gray)
print('len(kp)=', len(kp))
dst = cv2.drawKeypoints(gray, kp, None, color = (0, 0, 255))
cv2.imshow('dst', dst)

#3
kp = sorted(kp, key = lambda f: f.response, reverse= True)
filtered_kp = list(filter(lambda f:f.response > 50, kp))
filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))

kp, des = briskF.compute(gray, filtered_kp)
print('des.shape=', des.shape)
print('des=', des)

#4
dst2 = cv2.drawKeypoints(gray, filtered_kp, None, color= (0, 0, 255))
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x, y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst2, [box], True, (0, 255, 0), 2)
    cv2.circle(dst2, (round(x), round(y)), round(f.size / 2),
                (255, 0, 0), 2)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 9.9 다른 특징 검출기의 BRISK 디스크립터
import cv2
import numpy as np

#1 예제 9.2 거리에 따른 kp 제거
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def filteringByDistance(kp, distE=0.5):
    size = len(kp)
    mask = np.arange(1,size + 1).astype(np.bool8)   #all True
    for i, f1 in enumerate(kp):
        if not mask[i]:
            continue
        else: # True
            for j, f2 in enumerate(kp):
                if i == j:
                    continue
                if distance(f1, f2) < distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

#2
src = cv2.imread('0205.png\chessboard.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0.0)

fastF = cv2.FastFeatureDetector_create(threshold= 30)
mserF = cv2.MSER_create(10)
blobF = cv2.SimpleBlobDetector_create()
goodF = cv2.GFTTDetector_create(maxCorners= 20, minDistance= 10)

kp = fastF.detect(gray)
##kp = mserF.detect(gray)
##kp = blobF.detect(gray)
##kp = goodF.detect(gray)
print('len(kp)=', len(kp))

filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))
dst = cv2.drawKeypoints(gray, filtered_kp, None, color= (0, 0, 255))
cv2.imshow('dst', dst)

#3
briskF = cv2.BRISK_create()
filtered_kp, des = briskF.compute(gray, filtered_kp)
print('des.shape=', des.shape)
print('des=', des)

dst2 = cv2.drawKeypoints(gray, filtered_kp, None, color= (0, 0, 255))
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x, y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst2, [box], True, (0, 255, 0), 2)
    cv2.circle(dst2, (round(x), round(y)), round(f.size / 2),
                (255, 0, 0), 2)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 9.10 KAZE/AKAZE 특징 검출 및 디스크립터
import cv2
import numpy as np

#1 예제 9.2 거리에 따른 kp 제거
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def filteringByDistance(kp, distE=0.5):
    size = len(kp)
    mask = np.arange(1,size + 1).astype(np.bool8)   #all True
    for i, f1 in enumerate(kp):
        if not mask[i]:
            continue
        else: # True
            for j, f2 in enumerate(kp):
                if i == j:
                    continue
                if distance(f1, f2) < distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

#2 kazeF, akazeF 생성
src = cv2.imread('0205.png\chessboard.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0.0)

kazeF = cv2.KAZE_create()
akazeF = cv2.AKAZE_create()
kp = kazeF.detect(gray)
##kp = akazeF.detect(gray)
print('len(kp)=', len(kp))
dst = cv2.drawKeypoints(gray, kp, None, color= (0, 0, 255))
cv2.imshow('dst', dst)

#3
kp = sorted(kp, key = lambda f: f.response, reverse= True)
##filtered_kp = list(filter(lambda f:f.response > 0.01, kp))
filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))

kp, des = kazeF.compute(gray, filtered_kp)
##kp, des = akazeF.compute(gray, filtered_kp)
print('des.shape=', des.shape)
print('des.dtype=', des.dtype)
print('des=', des)

#4
dst2 = cv2.drawKeypoints(gray, filtered_kp, None, color= (0, 0, 255))
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x, y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst2, [box], True, (0, 255, 0), 2)
    cv2.circle(dst2, (round(x), round(y)), round(f.size / 2),
                (255, 0, 0), 2)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 9.11 SIFT 특징 검출 및 디스크립터
import cv2
import numpy as np

#1 예제 9.2 거리에 따른 kp 제거
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def filteringByDistance(kp, distE=0.5):
    size = len(kp)
    mask = np.arange(1,size + 1).astype(np.bool8)   #all True
    for i, f1 in enumerate(kp):
        if not mask[i]:
            continue
        else: # True
            for j, f2 in enumerate(kp):
                if i == j:
                    continue
                if distance(f1, f2) < distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

#2 cv2.SIFT_create()로 siftF생성
src = cv2.imread('0205.png\chessboard.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

##siftF = cv2.SIFT_create()
siftF = cv2.SIFT_create(edgeThreshold= 80)
kp = siftF.detect(gray)     #특징점 kp에 검출
print('len(kp)=', len(kp))

#3
kp = sorted(kp, key = lambda f: f.response, reverse= True)
##filtered_kp = list(filter(lambda f:f.response > 0.01, kp))
filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))

kp, des = siftF.compute(gray, filtered_kp)
print('des.shape=', des.shape)
print('des.dtype=', des.dtype)
print('des=', des)

#4
dst = cv2.drawKeypoints(src, filtered_kp, None, color= (0, 0, 255))
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x, y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst, [box], True, (0, 255, 0), 2)
    cv2.circle(dst, (round(x), round(y)), round(f.size / 2),
                (255, 0, 0), 2)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 9.12 이진 디스크립터 ORB, BRISK를 사용한 매칭과 투영 변환
import cv2
import numpy as np
#1
src1 = cv2.imread('0205.png\mario_1.jpg')       # 'cup1.jpg'
src2 = cv2.imread('0205.png\mario.jpg')       # 'cup2.jpg'
img1= cv2.cvtColor(src1, cv2.COLOR_BGR2GRAY)
img2= cv2.cvtColor(src2, cv2.COLOR_BGR2GRAY)

#2-1
orbF = cv2.ORB_create(nfeatures= 1000)
kp1, des1 = orbF.detectAndCompute(img1, None)
kp2, des2 = orbF.detectAndCompute(img2, None)

#2-2
##briskF = cv2.BRISK_create()
##kp1, des1 = briskF.detectAndCompute(img1, None)
##kp2, des2 = briskF.detectAndCompute(img2, None)

#3-1
bf = cv2.BFMatcher_create(cv2.NORM_HAMMING, crossCheck= True)
matches = bf.match(des1, des2)

#3-2
##flan = cv2.FlannBasedMatcher_create()
##matches = flan.match(np.float32(des1), np.float32(des2))

#4
matches = sorted(matches, key = lambda m: m.distance)
print('len(matches)=', len(matches))
for i, m in enumerate(matches[:3]):
    print('matches[{}]=(queryIdx:{}, trainIdx:{}, distance:{})'.format(i, m.queryIdx, m.trainIdx, m.distance))



minDist = matches[0].distance
good_matches = list(filter(
                        lambda m: m.distance < 5 * minDist, matches))
print('len(good_matches)=', len(good_matches))
if len(good_matches) < 5:
    print('sorry, too small good matches')
    exit()

dst = cv2.drawMatches(img1, kp1, img2, kp2, good_matches,
                      None, flags = 2)
cv2.imshow('dst', dst)

#5
src1_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches])
src2_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches])

#cv2.LMEDS
H, mask = cv2.findHomography(src1_pts, src2_pts, cv2.RANSAC, 3.0)
mask_matches = mask.ravel().tolist() #list(mask.flatten())

#6
h, w = img1.shape
pts = np.float32([[0, 0], [0, h - 1],[w - 1, h - 1],
                  [w-1, 0]]).reshape(-1,1,2)
pts2 = cv2.perspectiveTransform(pts, H)
src2 = cv2.polylines(src2,[np.int32(pts2)], True, (255, 0, 0), 2)

draw_params=dict(matchColor = (0, 255, 0), singlePointColor = None,
                 matchesMask = mask_matches, flags = 2)
dst2 = cv2.drawMatches(src1, kp1, src2, kp2, good_matches,
                       None, **draw_params)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 9.13 SIFT를 사용한 매칭과 투영 변환1:knnMatch()
import cv2
import numpy as np

#1
src1 = cv2.imread('0205.png\\racoon.jpg')       # 'cup1.jpg'
src2 = cv2.imread('0205.png\\aven.jpg')       # 'cup2.jpg'
img1= cv2.cvtColor(src1, cv2.COLOR_BGR2GRAY)
img2= cv2.cvtColor(src2, cv2.COLOR_BGR2GRAY)

#2
siftF = cv2.SIFT_create()
kp1, des1 = siftF.detectAndCompute(img1, None)
kp2, des2 = siftF.detectAndCompute(img2, None)

#3-1
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k = 2)
#3-2
##flan = cv2.FlannBasedMatcher_create()
##matches = flan.knnMatch(des1,des2, k = 2)

#3-3
print('len(matches)=', len(matches))
for i, m in enumerate(matches[:3]):
    for j, n in enumerate(m):
        print('matches[{}][{}]=(queryIdx:{}, trainIdx:{}, \distance:{})'.format(i, j, n.queryIdx, n.trainIdx, n.distance))


dst = cv2.drawMatchesKnn(img1, kp1, img2, kp2, matches, None, flags= 0)
##cv2.imshow('dst', dst)

#4: find good matches
nndrRatio = 0.45
good_matches = [f1 for f1, f2 in matches
                    if f1.distance < nndrRatio * f2.distance]

##good_matches = []
##for f1, f2 in matches: # k = 2
##    if f1.distance < nndrRatio*f2.distance:
##          good_matches.append(f1)

print('len(good_matches)=', len(good_matches))
if len(good_matches) < 5:
    print('sorry, too small good matches')
    exit()

#5
src1_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches])
src2_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches])

##
H, mask = cv2.findHomography(src1_pts, src2_pts, cv2.RANSAC, 2.0)
mask_matches = mask.ravel().tolist()    #list(mask.flatten())

#6
h,w = img1.shape
pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1],
                   [w - 1, 0]]).reshape(-1, 1, 2)
pts2 = cv2.perspectiveTransform(pts, H)
src2 = cv2.polylines(src2, [np.int32(pts2)], True, (255, 0, 0), 2)

draw_params=dict(matchColor = (0, 255, 0),
                 singlePointColor = None,
                 matchesMask = mask_matches, flags = 2)
dst2 = cv2.drawMatches(src1, kp1, src2, kp2, good_matches,
                       None, **draw_params)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 9.14 SIFT 특징 매칭:radiusMatch()
import cv2
import numpy as np

#1
src1 = cv2.imread('0205.png\\racoon.jpg')       # 'cup1.jpg'
src2 = cv2.imread('0205.png\\aven.jpg')       # 'cup2.jpg'
img1= cv2.cvtColor(src1, cv2.COLOR_BGR2GRAY)
img2= cv2.cvtColor(src2, cv2.COLOR_BGR2GRAY)

#2
siftF = cv2.SIFT_create()
kp1, des1 = siftF.detectAndCompute(img1, None)
kp2, des2 = siftF.detectAndCompute(img2, None)
print('len(kp1)=', len(kp1))
print('len(kp2)=', len(kp2))

#3
##bf = cv2.BFMatcher()
##matches = bf.radiusMatch(des1, des2, maxDistance = 50)
flan = cv2.FlannBasedMatcher_create()
matches = flan.radiusMatch(des1,des2, maxDistance = 50) #200
#print('# of matches =', len(np.nonzero(
#                               np.array(matches, dtype = object))[0]))
#4
def draw_key2image(kp, img):
    x, y = kp.pt
    size = kp.size
    rect = ((x, y), (size, size), kp.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(img, [box], True, (0, 255, 0), 2)
    cv2.circle(img, (round(x), round(y)), round(size/2),
                (255, 0, 0), 2)
##      return img

for i, radius_match in enumerate(matches):
    if len(radius_match) != 0:
        print('i =', i)
        print('len(matches[{}])={}',format(i, len(matches[i])))
##                                            len(radius_match)                    
        src1c = src1.copy()
        draw_key2image(kp1[radius_match[0].queryIdx], src1c)
        src2c = src2.copy()
        for m in radius_match:
            draw_key2image(kp2[m.trainIdx], src2c)
        dst = cv2.drawMatches(src1c, kp1, src2c, kp2, radius_match,
                                None, flags= 2)
        cv2.imshow('dst', dst)
        cv2.waitKey()
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 9.15 SIFT 특징 매칭(radiusMatch)과 투영 변환 2
import cv2
import numpy as np

#1
src1 = cv2.imread('0205.png\\racoon.jpg')       # 'cup1.jpg'
src2 = cv2.imread('0205.png\\aven.jpg')       # 'cup2.jpg'
img1= cv2.cvtColor(src1, cv2.COLOR_BGR2GRAY)
img2= cv2.cvtColor(src2, cv2.COLOR_BGR2GRAY)

#2
siftF = cv2.SIFT_create()
kp1, des1 = siftF.detectAndCompute(img1, None)
kp2, des2 = siftF.detectAndCompute(img2, None)
print('len(kp1)={}, len(kp2)={}'.format(len(kp1), len(kp2)))

#3
distT = 200         #500
##bf = cv2.BFMatcher()
##matches = bf.radiusMatch(des1, des2, maxDistance = distT)
flan = cv2.FlannBasedMatcher_create()
matches = flan.radiusMatch(des1,des2, maxDistance = distT) 
print('len(matches)=', len(matches))

#4
good_matches = []
for i, radius_match in enumerate(matches):
#4-1
##  if len(radius_match) !=0:
##      sort_match = sorted(radius_match, key = lanbda m:
# m.distance)
##      good_matches.append(sort_match[0])
#4-2
    if len(radius_match) != 0:
        for m in radius_match:
            if m.distance<100:  #filter by distance
                good_matches.append(m)
print('len(good_matches)-', len(good_matches))
##dst2 = cv2.drawMatches(img1, kp1, img2, kp2,
#                         good_matches, None, flags = 2)
####cv2.imshow('dst2', dst2)

#5
src1_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches])
src2_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches])

##
H, mask = cv2.findHomography(src1_pts, src2_pts, cv2.RANSAC, 3.0)
mask_matches = mask.ravel().tolist()        # list(mask.flatten())

#6
h,w = img1.shape
pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1],
                   [w - 1, 0]]).reshape(-1, 1, 2)
pts2 = cv2.perspectiveTransform(pts, H)
src2 = cv2.polylines(src2, [np.int32(pts2)], True, (255, 0, 0), 2)

draw_params=dict(matchColor = (0, 255, 0), singlePointColor = None,
                 matchesMask = mask_matches, flags = 2)
dst3 = cv2.drawMatches(src1, kp1, src2, kp2, good_matches,
                       None, **draw_params)
cv2.imshow('dst3', dst3)

cv2.waitKey()
cv2.destroyAllWindows()